# Decorators Application — Logger & Stacked Decorators (Advanced Problems + Solutions)

This notebook focuses on **production-grade logging decorators** and **stacked decorator behavior**:
- structured logging (no `print` by default)
- consistent UTC timestamps
- exception logging (without swallowing errors)
- argument redaction (PII / secrets)
- async support
- decorator stacking order and practical implications (auth vs log)
- preserving function metadata and `__wrapped__`


In [1]:
from __future__ import annotations

import asyncio
import functools
import inspect
import logging
import time
from dataclasses import dataclass
from datetime import datetime, timezone
from typing import Any, Callable


## Logging setup for the notebook

We configure a simple console logger to make examples visible.
In production, you'd usually configure handlers/formatters elsewhere.


In [2]:
logger = logging.getLogger("decorators_demo")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)sZ %(levelname)s %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Ensure UTC timestamps in log output (Formatter uses local time by default)
logging.Formatter.converter = time.gmtime


## Exercise 1 — A best-practice `logged` decorator factory (structured + redaction)

Implement `logged` as a decorator factory:

```python
@logged(logger=logger, level=logging.INFO, event="call", redact_keys={"password"})
def f(user, password):
    ...
```

Requirements:
- Uses `functools.wraps`.
- Logs a structured message with:
  - event name
  - function name
  - UTC timestamp (ISO-8601)
  - args/kwargs (optional, but supported)
- Supports `redact_keys` for kwargs to avoid logging secrets.
- Does **not** swallow exceptions.
- Adds `wrapper.last_call` dict with the most recent call metadata.

Best practices:
- Default to not logging args (`log_args=False`) to reduce noise and risk.
- Prefer `logging` over `print`.


In [3]:
# --- Solution (Exercise 1) ---

def _utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def _safe_repr(x: Any, max_len: int) -> str:
    s = repr(x)
    if len(s) <= max_len:
        return s
    return s[: max(0, max_len - 1)] + "…"

def logged(*,
           logger: logging.Logger,
           level: int = logging.INFO,
           event: str = "call",
           log_args: bool = False,
           max_repr: int = 120,
           redact_keys: set[str] | None = None,
           redact_value: str = "<redacted>"):
    """Log calls to a function.

    - Structured logging (message is a compact key=value string).
    - Optional arg logging with redaction.
    - Exposes wrapper.last_call.
    """
    if redact_keys is None:
        redact_keys = set()

    def decorate(fn: Callable):
        last_call: dict[str, Any] | None = None

        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            nonlocal last_call
            call_meta: dict[str, Any] = {
                "event": event,
                "fn": fn.__name__,
                "ts": _utc_now_iso(),
            }

            if log_args:
                safe_kwargs = {
                    k: (redact_value if k in redact_keys else v)
                    for k, v in kwargs.items()
                }
                call_meta["args"] = [_safe_repr(a, max_repr) for a in args]
                call_meta["kwargs"] = {
                    k: _safe_repr(v, max_repr) for k, v in safe_kwargs.items()
                }

            # Log before execution (useful for auth-blocked calls when stacked appropriately)
            msg_parts = [
                f"event={call_meta['event']}",
                f"fn={call_meta['fn']}",
                f"ts={call_meta['ts']}",
            ]
            if log_args:
                msg_parts.append(f"args={call_meta['args']}")
                msg_parts.append(f"kwargs={call_meta['kwargs']}")
            logger.log(level, " ".join(msg_parts))

            last_call = call_meta
            wrapper.last_call = last_call  # type: ignore[attr-defined]
            return fn(*args, **kwargs)

        wrapper.last_call = last_call  # type: ignore[attr-defined]
        return wrapper

    return decorate


## Exercise 2 — A `timed` decorator that is stack-friendly

Implement `timed` that:
- uses `perf_counter`
- stores `last_elapsed`
- does not print
- optionally logs timing via a provided logger

Also ensure it preserves metadata with `wraps`.


In [4]:
# --- Solution (Exercise 2) ---

def timed(*,
          logger: logging.Logger | None = None,
          level: int = logging.INFO,
          event: str = "timing"):
    def decorate(fn: Callable):
        last_elapsed = 0.0

        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            nonlocal last_elapsed
            t0 = time.perf_counter()
            try:
                return fn(*args, **kwargs)
            finally:
                last_elapsed = time.perf_counter() - t0
                wrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
                if logger is not None:
                    logger.log(
                        level,
                        f"event={event} fn={fn.__name__} elapsed_s={last_elapsed:.9f}",
                    )

        wrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
        return wrapper

    return decorate


## Exercise 3 — Stacking order matters (log-before vs log-after auth)

Create an `authorize` decorator factory:

```python
@authorize(lambda user: "admin" in user.roles)
def endpoint(user):
    ...
```

Requirements:
- If predicate fails, raise `PermissionError`.
- Preserve metadata.

Then demonstrate two endpoints:

1)
```python
@logged(...)
@authorize(...)
def e1(...):
    ...
```

2)
```python
@authorize(...)
@logged(...)
def e2(...):
    ...
```

Show (via logs) that **e1 logs even unauthorized calls**, while **e2 logs only authorized calls**.


In [5]:
# --- Solution (Exercise 3) ---

def authorize(predicate: Callable[..., bool], *, error: str = "unauthorized"):
    if not callable(predicate):
        raise TypeError("predicate must be callable")

    pred_sig = inspect.signature(predicate)
    params = pred_sig.parameters

    has_var_kw = any(p.kind == inspect.Parameter.VAR_KEYWORD for p in params.values())
    has_var_pos = any(p.kind == inspect.Parameter.VAR_POSITIONAL for p in params.values())

    pos_kinds = {inspect.Parameter.POSITIONAL_ONLY, inspect.Parameter.POSITIONAL_OR_KEYWORD}
    max_pos = None if has_var_pos else sum(1 for p in params.values() if p.kind in pos_kinds)

    allowed_kw = {
        name for name, p in params.items()
        if p.kind in {inspect.Parameter.POSITIONAL_OR_KEYWORD, inspect.Parameter.KEYWORD_ONLY}
    }

    def decorate(fn: Callable):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            args2 = args if max_pos is None else args[:max_pos]
            kwargs2 = kwargs if has_var_kw else {k: v for k, v in kwargs.items() if k in allowed_kw}

            ok = predicate(*args2, **kwargs2)
            if not ok:
                raise PermissionError(error)
            return fn(*args, **kwargs)

        return wrapper

    return decorate

@dataclass
class User:
    name: str
    roles: set[str]


In [6]:
# Demo endpoints for stacking order

is_admin = lambda user: "admin" in user.roles

@logged(logger=logger, event="endpoint_call", log_args=True, redact_keys={"token"})
@authorize(is_admin)
def e1(user: User, *, token: str = ""):
    return f"ok:{user.name}"

@authorize(is_admin)
@logged(logger=logger, event="endpoint_call", log_args=True, redact_keys={"token"})
def e2(user: User, *, token: str = ""):
    return f"ok:{user.name}"

admin = User("alice", {"admin"})
guest = User("bob", set())


## Exercise 4 — Exception-aware logging decorator (log failures too)

Implement `logged_exceptions`:
- Logs successful calls at INFO.
- Logs exceptions at ERROR including exception type.
- Re-raises the original exception.
- Preserves metadata.

Best practice: log exception information with `logger.exception(...)`.


In [7]:
# --- Solution (Exercise 4) ---

def logged_exceptions(*, logger: logging.Logger, event: str = "call"):
    def decorate(fn: Callable):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            ts = _utc_now_iso()
            try:
                result = fn(*args, **kwargs)
                logger.info(f"event={event} status=ok fn={fn.__name__} ts={ts}")
                return result
            except Exception as e:
                # Includes traceback
                logger.exception(
                    f"event={event} status=error fn={fn.__name__} ts={ts} exc_type={type(e).__name__}"
                )
                raise
        return wrapper
    return decorate


## Exercise 5 — Async-safe logging (works for `async def`)

Implement `logged_any` which wraps both sync and async functions.

- If async, `await` the function.
- Log start + end with elapsed time.
- Preserve metadata.
- Expose `last_elapsed`.


In [8]:
# --- Solution (Exercise 5) ---

def logged_any(*, logger: logging.Logger, event: str = "call", level: int = logging.INFO):
    def decorate(fn: Callable):
        last_elapsed = 0.0

        if asyncio.iscoroutinefunction(fn):
            @functools.wraps(fn)
            async def awrapper(*args, **kwargs):
                nonlocal last_elapsed
                ts = _utc_now_iso()
                logger.log(level, f"event={event} phase=start fn={fn.__name__} ts={ts}")
                t0 = time.perf_counter()
                try:
                    return await fn(*args, **kwargs)
                finally:
                    last_elapsed = time.perf_counter() - t0
                    awrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
                    logger.log(
                        level,
                        f"event={event} phase=end fn={fn.__name__} elapsed_s={last_elapsed:.9f} ts={_utc_now_iso()}",
                    )

            awrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
            return awrapper

        @functools.wraps(fn)
        def swrapper(*args, **kwargs):
            nonlocal last_elapsed
            ts = _utc_now_iso()
            logger.log(level, f"event={event} phase=start fn={fn.__name__} ts={ts}")
            t0 = time.perf_counter()
            try:
                return fn(*args, **kwargs)
            finally:
                last_elapsed = time.perf_counter() - t0
                swrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
                logger.log(
                    level,
                    f"event={event} phase=end fn={fn.__name__} elapsed_s={last_elapsed:.9f} ts={_utc_now_iso()}",
                )

        swrapper.last_elapsed = last_elapsed  # type: ignore[attr-defined]
        return swrapper

    return decorate


## Exercise 6 — Prove metadata survives stacking (`inspect.signature`, `__wrapped__`)

Create a function with annotations and defaults, decorate it with both `logged` and `timed`, and verify:
- `__name__` remains the original name
- `inspect.signature` matches the original signature

Best practice: use `wraps` everywhere so inspection tools can follow `__wrapped__`.


In [9]:
# --- Solution (Exercise 6) ---

@timed(logger=logger, event="timing")
@logged(logger=logger, event="call", log_args=True, redact_keys={"secret"})
def compute(x: int, y: int = 10, *, secret: str = "pw") -> int:
    """Compute x + y."""
    return x + y


# Verification / Tests

Run this cell to see behavior and validate requirements.


In [10]:
# --- Tests / Demos ---

# Exercise 1: redaction + last_call
@logged(logger=logger, event="demo", log_args=True, redact_keys={"password"})
def login(user: str, *, password: str):
    return f"welcome:{user}"

assert login("neo", password="matrix") == "welcome:neo"
assert login.__name__ == "login"
assert login.last_call["fn"] == "login"  # type: ignore[index]
assert "password" in login.last_call.get("kwargs", {})  # type: ignore[union-attr]
assert (
    login.last_call["kwargs"]["password"] == "'<redacted>'"
    or "<redacted>" in login.last_call["kwargs"]["password"]
)  # type: ignore[index]

# Exercise 3: stacking behavior
try:
    e1(guest, token="SECRET")
    raise AssertionError("Expected PermissionError")
except PermissionError:
    pass

try:
    e2(guest, token="SECRET")
    raise AssertionError("Expected PermissionError")
except PermissionError:
    pass

assert e1(admin, token="SECRET") == "ok:alice"
assert e2(admin, token="SECRET") == "ok:alice"

# Exercise 4: exception logging does not swallow
@logged_exceptions(logger=logger, event="boom")
def will_fail(x: int):
    return 10 // x

assert will_fail(2) == 5
try:
    will_fail(0)
    raise AssertionError("Expected ZeroDivisionError")
except ZeroDivisionError:
    pass

# Exercise 5: async
@logged_any(logger=logger, event="async_demo")
async def slow_add(a: int, b: int) -> int:
    await asyncio.sleep(0)
    return a + b

async def _run_async():
    v = await slow_add(2, 3)
    assert v == 5
    assert slow_add.last_elapsed >= 0.0

await _run_async()

# Exercise 6: metadata under stacking
assert compute.__name__ == "compute"
assert "Compute" in (compute.__doc__ or "")
sig = inspect.signature(compute, globals=globals(), locals=locals(), eval_str=True)
assert str(sig) == "(x: int, y: int = 10, *, secret: str = 'pw') -> int"

assert compute(5, 7, secret="TOP") == 12
assert hasattr(compute, "last_elapsed")

print("All checks passed ✅")


2026-02-06 19:10:40,269Z INFO event=demo fn=login ts=2026-02-06T19:10:40.269190+00:00 args=["'neo'"] kwargs={'password': "'<redacted>'"}
2026-02-06 19:10:40,272Z INFO event=endpoint_call fn=e1 ts=2026-02-06T19:10:40.272312+00:00 args=["User(name='bob', roles=set())"] kwargs={'token': "'<redacted>'"}
2026-02-06 19:10:40,275Z INFO event=endpoint_call fn=e1 ts=2026-02-06T19:10:40.275343+00:00 args=["User(name='alice', roles={'admin'})"] kwargs={'token': "'<redacted>'"}
2026-02-06 19:10:40,277Z INFO event=endpoint_call fn=e2 ts=2026-02-06T19:10:40.277077+00:00 args=["User(name='alice', roles={'admin'})"] kwargs={'token': "'<redacted>'"}
2026-02-06 19:10:40,278Z INFO event=boom status=ok fn=will_fail ts=2026-02-06T19:10:40.278726+00:00
2026-02-06 19:10:40,280Z ERROR event=boom status=error fn=will_fail ts=2026-02-06T19:10:40.280104+00:00 exc_type=ZeroDivisionError
Traceback (most recent call last):
  File "C:\Users\user1\AppData\Local\Temp\ipykernel_4072\1399398358.py", line 9, in wrapper
 

All checks passed ✅
